### Nối Colab vô Drive 

In [137]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Đi đến thư mục chứa data

In [138]:
cd /content/drive/My Drive/Colab Notebooks/ProtonX

/content/drive/My Drive/Colab Notebooks/ProtonX


In [0]:
# !wget --no-check-certificate \
#   https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
#   -O ./cats_and_dogs_filtered.zip

In [0]:
# main_folder = './'
# !unzip -q cats_and_dogs_filtered.zip -d $main_folder

In [141]:
cd cats_and_dogs_filtered

/content/drive/My Drive/Colab Notebooks/ProtonX/cats_and_dogs_filtered


In [0]:
train_folder = './train'

In [0]:
valid_folder = './validation'

### [TODO 1] Xây dựng mô hình Deep Learning

In [0]:
train_dog_dir = 'train/dogs'
train_cat_dir = 'train/cats'
valid_dog_dir = 'validation/dogs'
valid_cat_dir = 'validation/cats'

In [0]:
# Data Preprocessing

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale = 1.0/255.0, rotation_range= 0.2, zoom_range = 0.1,
                                    # featurewise_center=True, featurewise_std_normalization=True,
                                    width_shift_range = 0.2, height_shift_range = 0.2,
                                    horizontal_flip = True)

valid_generator = ImageDataGenerator(rescale = 1.0/255.0)

In [0]:
size_image = (128,128)
batch_size = 64

In [147]:
train_generator_data = train_generator.flow_from_directory(
    train_folder, target_size = size_image , batch_size = batch_size, shuffle = True,
    class_mode = 'binary')

valid_generator_data = valid_generator.flow_from_directory(
    valid_folder, target_size = size_image, batch_size = batch_size,
    class_mode = 'binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Conv2D, Activation, Dropout
from tensorflow.keras.layers import Flatten, Input, Dense, concatenate
from tensorflow.keras.optimizers import RMSprop, Adam, SGD

In [0]:
model = Sequential()

model.add(Conv2D(32,(3,3), padding = 'same', input_shape = size_image + (3,), kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3), padding = 'same', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(tf.keras.layers.Dropout(rate = 0.25))

model.add(Conv2D(64,(3,3), padding = 'same', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3), padding = 'same', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(tf.keras.layers.Dropout(rate = 0.25))

model.add(Conv2D(128,(3,3), padding = 'same', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3), padding = 'same', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(tf.keras.layers.Dropout(rate = 0.25))

model.add(Conv2D(256,(3,3), padding = 'same', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3), padding = 'same', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(tf.keras.layers.Dropout(rate = 0.25))

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation = 'sigmoid'))

In [150]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_64 (Conv2D)           (None, 128, 128, 32)      896       
_________________________________________________________________
activation_64 (Activation)   (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization_72 (Batc (None, 128, 128, 32)      128       
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 128, 128, 32)      9248      
_________________________________________________________________
activation_65 (Activation)   (None, 128, 128, 32)      0         
_________________________________________________________________
batch_normalization_73 (Batc (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 64, 64, 32)       

In [0]:
model.compile(
    optimizer = RMSprop(learning_rate = 0.001, momentum = 0.001),
    # optimizer = Adam(),
    loss = 'binary_crossentropy',
    metrics = ['accuracy'])

### [TODO 2] Thực hiện training

In [0]:
import tensorflow as tf

DESIRED_ACCURACY = 0.9

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>DESIRED_ACCURACY):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [153]:
history = model.fit(
    train_generator_data,
    steps_per_epoch = 250, 
    epochs = 50, shuffle = True,
    validation_data = valid_generator_data,
    validation_steps = 100,
    callbacks=[callbacks])

Epoch 1/50
250/250 [==============================] - 136s 543ms/step - loss: 1.3537 - accuracy: 0.6180 - val_loss: 1.1444 - val_accuracy: 0.5006
Epoch 2/50
250/250 [==============================] - 134s 538ms/step - loss: 0.7516 - accuracy: 0.6954 - val_loss: 0.7313 - val_accuracy: 0.6333
Epoch 3/50
250/250 [==============================] - 134s 535ms/step - loss: 0.6138 - accuracy: 0.7500 - val_loss: 0.9519 - val_accuracy: 0.5806
Epoch 4/50
250/250 [==============================] - 134s 535ms/step - loss: 0.5428 - accuracy: 0.7837 - val_loss: 0.5590 - val_accuracy: 0.7754
Epoch 5/50
250/250 [==============================] - 134s 535ms/step - loss: 0.4880 - accuracy: 0.8207 - val_loss: 0.5217 - val_accuracy: 0.8039
Epoch 6/50
250/250 [==============================] - 135s 538ms/step - loss: 0.4326 - accuracy: 0.8466 - val_loss: 0.4783 - val_accuracy: 0.8215
Epoch 7/50
250/250 [==============================] - 135s 540ms/step - loss: 0.3920 - accuracy: 0.8713 - val_loss: 0.5648 -

### [TODO 3] Thực hiện load file và dự đoán

In [0]:
model.save('catdog.h5')

In [0]:
saved_model =  tf.keras.models.load_model('catdog.h5')

In [156]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = './' + fn
  img = image.load_img(path, target_size = (128,128))
  x = image.img_to_array(img)/255.0

  x = np.expand_dims(x, axis = 0)

  image = np.vstack([x])

  classes = saved_model.predict(image, batch_size=10)

  if classes[0] > 0.5:
    print(fn+ ' is a dog.')
  else:
    print(fn+ ' is a cat.')

Using TensorFlow backend.


MessageError: ignored

### Hiển thị độ chính xác của mô hình

In [0]:
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc      = history.history[     'accuracy' ]
val_acc  = history.history[ 'val_accuracy' ]
loss     = history.history[    'loss' ]
val_loss = history.history['val_loss' ]

epochs   = range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot  ( epochs,     acc )
plt.plot  ( epochs, val_acc )
plt.title ('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot  ( epochs,     loss )
plt.plot  ( epochs, val_loss )
plt.title ('Training and validation loss'   )